# Imports

In [ ]:
from scripts.datasets import simulation_classes
from scripts.bmlp import ScBMLPClassifier, Config

import scanpy as sc
import numpy as np
import plotly.express as px
import pandas as pd
import einops
import torch

# Set params

In [2]:
DEVICE = "cpu"  # faster than mps...

# Load data

In [3]:
adata, train_dataset, val_dataset, test_dataset = simulation_classes(device=DEVICE, n_cell_types=3, class_key="cell_type")

NameError: name 'simulation_classes' is not defined

In [ ]:
n_cells, n_genes = adata.shape
n_cell_types = adata.obs["cell_type"].nunique()

## Visualize

In [ ]:
fig = px.scatter(
    x=adata.obsm["X_pca"][:, 0],
    y=adata.obsm["X_pca"][:, 1],
    color=adata.obs["cell_type"],
    title="Myeloid progenitor differentiation data",
    width=600,
    height=600,
)
fig.update_traces(marker=dict(size=5))
fig.show()

# Train model

In [ ]:
d_hidden = 64
n_epochs = 100
lr = 1e-5

In [ ]:
cfg = Config(
    d_input=n_genes,
    d_hidden=d_hidden,
    d_output=n_cell_types,
    n_epochs=n_epochs,
    lr=lr,
    device=DEVICE,
)
model = ScBMLPClassifier(cfg)
train_losses, val_losses = model.fit(train_dataset, val_dataset)

Training for 100 epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Training for 100 epochs: 100%|██████████| 100/100 [00:12<00:00,  7.86it/s, train_acc=1.0000, train_loss=0.0066, val_acc=0.8949, val_loss=0.2756]


In [ ]:
# Combine train and val losses into a single plot
loss_df = pd.DataFrame({
    'Epoch': list(range(len(train_losses))) + list(range(len(val_losses))),
    'Loss': train_losses + val_losses,
    'Type': ['Train'] * len(train_losses) + ['Validation'] * len(val_losses)
})

px.line(loss_df, x='Epoch', y='Loss', color='Type', 
        title='Training and Validation Loss', 
        labels={'Loss': 'Loss', 'Epoch': 'Epoch'}).show()